<a href="https://colab.research.google.com/github/neoyipeng2018/happier/blob/master/Happier_v0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
!curl https://course.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   321  100   321    0     0   1629      0 --:--:-- --:--:-- --:--:--  1637
Updating fastai...
Done.


In [0]:
from fastai.collab import *
from fastai.tabular import *

## Basic linear model
Using embedding size of 50

In [72]:
!git clone --recursive https://github.com/neoyipeng2018/happier.git 

Cloning into 'happier'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [74]:
ratings = pd.read_csv('happier/sample.csv')
ratings.head(2)

,userId,activityId,rating,timestamp
0,1,1,3,1
1,2,1,4,2


In [0]:
data = CollabDataBunch.from_df(ratings, seed=42, bs=10)

In [0]:
y_range = [0,5.5]

In [0]:
learn = collab_learner(data, n_factors=50, y_range=y_range)

In [84]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(3, 50)
  (i_weight): Embedding(11, 50)
  (u_bias): Embedding(3, 1)
  (i_bias): Embedding(11, 1)
)

In [82]:
learn.fit_one_cycle(12, 1e-1)

epoch,train_loss,valid_loss
1,2.322735,1.219804
2,2.410240,1.250920
3,2.172191,1.256604
4,2.112640,1.363037
5,1.784759,1.903173
6,1.572342,1.392496
7,1.393049,0.823289
8,1.242996,0.822927
9,1.116788,1.117585
10,1.020988,1.086690


In [85]:
df = pd.DataFrame(data={'userId':[2],'activityId':[10],'rating':[0],'timestamp':[0]})
df

,activityId,rating,timestamp,userId
0,10,0,0,2


In [86]:
min(learn.predict(df.iloc[0])[1],5)

5

## Basic Neural Net model
Embedding layers followed by a 2 Linear/Relu/Batchnorm layer blocks.

In [0]:
learn = collab_learner(data, y_range=y_range, use_nn=True, emb_szs={'userId':50, 'activityId':50}, layers = [100,50], emb_drop=0.5)

In [89]:
learn.model

EmbeddingNN(
  (embeds): ModuleList(
    (0): Embedding(3, 50)
    (1): Embedding(11, 50)
  )
  (emb_drop): Dropout(p=0.5)
  (bn_cont): BatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU(inplace)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=100, out_features=50, bias=True)
    (4): ReLU(inplace)
    (5): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [94]:
learn.fit_one_cycle(12, 1e-1)

epoch,train_loss,valid_loss
1,3.161116,1.023955
2,2.255918,1.210855
3,2.308122,1.792996
4,2.067180,2.826253
5,1.861662,3.918941
6,1.620632,4.093886
7,1.432236,3.591331
8,1.365248,2.778302
9,1.333323,1.985919
10,1.273898,1.640603


In [95]:
df = pd.DataFrame(data={'userId':[2],'activityId':[10],'rating':[0],'timestamp':[0]})
df

,activityId,rating,timestamp,userId
0,10,0,0,2


In [96]:
min(learn.predict(df.iloc[0])[1],5)

5

### Extra: How someone tackled the cold start problem - Create a meta model
At my place of work we have the following approach to a cold-start problem:

For context, I am at a financial institution and we have trained a collaborative filtering model on data purchased from a third party which consists of ~40k businesses detailing their cash management needs. Specifically, it might look like business Y has reported it uses lock-box, fraud-management, equity-management, etc. (from a list of about 40 products).

How we use this to build a recommender for our own customers (who were not part of the model training process): the model works by embedding each unique user and product into a space of some fixed dimension and modeling the probability as a dot product or perhaps shallow neural network. So given a new user, if we knew where they stood in the embedding dimension we would be able to apply our model and say how likely does it think this user is to want product X.

We build a second model (as Jeremy has suggested) using auxiliary data, things like sales volume, number of employees, SIC codes (what ‘kind’ of business it is) and train this model with the following 40k datapoints: we know these features about the businesses in our purchased dataset, and we build a regressor that maps from these “identifying features” to its position in embedding dimension with RMSE error. Once we are satisfied we can situate a new client reasonably well based on these identifying features, we can take a new user, apply model 2 to situate them in embedding space, and then apply the original collaborative filtering model.

Happy to discuss if someone is curious!

Link: https://forums.fast.ai/t/lesson-4-advanced-discussion/30319/37